In [87]:
import importlib
import inv
import labs
import vote

importlib.reload(labs)
importlib.reload(inv)
importlib.reload(vote)

from vote import *
from inv import *
from labs import *

label2id, id2label = make_doc_class_mapper('/data/ephemeral/home/dataset/dtc/doc_classes.json')
id2kor = make_id2kor(id2label)

# load history

In [2]:
path_csvs = grep_files('./history/', exts=['csv'])

submits = []
for p in path_csvs: 
    df = pd.read_csv(p)
    submits.append(df)

0it [00:00, ?it/s]

# Voting
- TODO
  - fixed 된 것에서 cnv 대비 다른 것 
  - cnv가 약점(1, 3, 4, 7, 13, 14) 로 예상한 리스트는 llv 따름
  - cnv가 비약점 클래스에서 90 이상인 것은 고정

In [17]:
misses = read_list_from_txt('./inconsist_list.txt')

In [3]:
cnv_probs = read_list_from_txt('./probs/cnv2-easy-rot-dice-loss.txt')
llv_probs = read_list_from_txt('./probs/llv3-dice-loss-sub-classes.txt')

In [42]:
def load_probs(path, misses, exclude_weak=True):
    weaks = [1, 3, 4, 7, 13, 14]
    name2probs = read_list_from_txt(path)
    rows = []
    for line in name2probs:
        item = eval(line)
        filename = item[0][:-6] + '.jpg'
        probs = np.array(item[1])
        label_id = np.argmax(probs)
        score = probs[label_id]
        if exclude_weak and int(label_id) in weaks:
            continue
        if filename in misses and score > 0.80:
            rows.append((filename, label_id, score))
    return rows

In [43]:
name2probs = load_probs('./probs/cnv2-easy-rot-dice-loss.txt', misses=misses)

In [64]:
# 1차 비약점 cnv 확신답 모음
answers = name2probs[:]
checked_codes = [a[0] for a in answers]

In [65]:
for line in llv_probs:
    item = eval(line)
    filename = item[0]
    probs = np.array(item[1])
    label_id = np.argmax(probs)
    score = probs[label_id]
    if filename in checked_codes:
        continue
    if filename in misses:
        answers.append((filename, label_id, score))

In [66]:
len(answers)

409

In [70]:
answers[0]

('77a1ccb7d5125c15.jpg', 11, 0.9976)

# 불일치 채우기

In [80]:
df = pd.read_csv('./fixed.csv')

In [81]:
for item in answers:
    matched = df.index[df['ID'] == item[0]]
    idx = matched[0]
    df.loc[idx, 'target'] = int(item[1])

In [83]:
df.to_csv('voted.csv', index=False)

# 보기

In [84]:
ds_dir_path = '/data/ephemeral/home/dataset/docsy-deskew/'
miss_paths = [os.path.join(ds_dir_path, m) for m in misses]

In [88]:
inv = ImageNavigator(miss_paths, [df], id2kor)